# Introduction & 0/1 Knapsack (medium)

Apply advanced techniques of Memoization and Bottom-Up Dynamic Programming to solve the overlapping subproblems.

Given the weights and profits of 'N' items, we are asked to put these items in a knapsack with a capacity 'C'. The goal is to get the maximum profit out of the knapsack items. Each item can only be selected once, as we don’t have multiple quantities of any item.

Let’s take Merry’s example, who wants to carry some fruits in the knapsack to get maximum profit. Here are the weights and profits of the fruits:

Items: { Apple, Orange, Banana, Melon }<br>
Weights: { 2, 3, 1, 4 }<br>
Profits: { 4, 5, 3, 7 }<br>
Knapsack capacity: 5<br>

Let’s try to put various combinations of fruits in the knapsack, such that their total weight is not more than 5:

Apple + Orange (total weight 5) => 9 profit<br>
Apple + Banana (total weight 3) => 7 profit<br>
Orange + Banana (total weight 4) => 8 profit<br>
Banana + Melon (total weight 5) => 10 profit<br>

This shows that Banana + Melon is the best combination as it gives us the maximum profit, and the total weight does not exceed the capacity.

### Problem Statement
Given two integer arrays to represent weights and profits of 'N' items, we need to find a subset of these items which will give us maximum profit such that their cumulative weight is not more than a given number 'C'. Each item can only be selected once, which means either we put an item in the knapsack or we skip it.<br>

**IMPORTANT**: https://www.educative.io/courses/grokking-the-coding-interview/gkZNLjV2kBk

### Brute Force
Try all combinations of the given items to choose the one with maximum profit and a weight that doesn't exceed 'C'.

In [1]:
def solve_knapsack(profits, weights, capacity):
    return knapsack_recursive(profits, weights, capacity, 0)

def knapsack_recursive(profits, weights, capacity, currentIndex):
    # base check
    if capacity <= 0 or currentIndex >= len(profits):
        return 0
    
    # recursive call after choosing the element at the currentIndex
    # if the weight of the element at currentIndex exceeds the capacity, we  shouldn't process this
    profit1 = 0
    if weights[currentIndex] <= capacity:
        profit1 = profits[currentIndex] + knapsack_recursive(
            profits, weights, capacity - weights[currentIndex], currentIndex + 1)
    
    # recursive call after excluding the element at the currentIndex
    profit2 = knapsack_recursive(profits, weights, capacity, currentIndex + 1)
    
    return max(profit1, profit2)

def main():
    print(solve_knapsack([1, 6, 10, 16], [1, 2, 3, 5], 7))
    print(solve_knapsack([1, 6, 10, 16], [1, 2, 3, 5], 6))

main()

22
17


**Time Complexity**: $O(2^n)$, where 'n' represents the total number of items. This can also be confirmed from the above recursion tree. As we can see, we will have a total of '31' recursive calls – calculated through $(2^n) + (2^n) - 1$, which is asymptotically equivalent to $O(2^n)$.<br>
**Space Complexity**: $O(n)$ to store the recursion stack.

### Top-down Dynamic Programming with Memoization

Memoization is when we store the results of all the previously solved sub-problems and return the results from memory if we encounter a problem that has already been solved.

Since we have two changing values (*capacity* and *currentIndex*) in our recursive function *knapsackRecursive()*, we can use a two-dimensional array to store the results of all the solved sub-problems. As mentioned above, we need to store results for every sub-array (i.e., for every possible index 'i') and every possible capacity 'c'.

In [5]:
def solve_knapsack(profits, weights, capacity):
    # create a two dimensional array for Memoization, each element is initialized to '-1'
    dp = [[-1 for x in range(capacity+1)] for y in range(len(profits))]
    return knapsack_recursive(dp, profits, weights, capacity, 0)

def knapsack_recursive(dp, profits, weights, capacity, currentIndex):
    # base check
    if capacity <= 0 or currentIndex >= len(profits):
        return 0
    
    # if we have already solved a similar problem, return the result from memory
    if dp[currentIndex][capacity] != -1:
        return dp[currentIndex][capacity]
    
    # recursive call after choosing the element at the currentIndex
    # if the weight of the element at currentIndex exceeds the capacity, we
    # shouldn't process this
    profit1 = 0
    if weights[currentIndex] <= capacity:
        profit1 = profits[currentIndex] + knapsack_recursive(
            dp, profits, weights, capacity - weights[currentIndex], currentIndex + 1)
    
    # recursive call after excluding the element at the currentIndex
    profit2 = knapsack_recursive(
        dp, profits, weights, capacity, currentIndex + 1)
    
    dp[currentIndex][capacity] = max(profit1, profit2)
    return dp[currentIndex][capacity]

def main():
    print(solve_knapsack([1, 6, 10, 16], [1, 2, 3, 5], 7))
    print(solve_knapsack([1, 6, 10, 16], [1, 2, 3, 5], 6))

main()

22
17


**Time Complexity**: $O(N * C)$. Since our memoization array dp[profits.length][capacity+1] stores the results for all subproblems, we can conclude that we will not have more than $N*C$ subproblems (where 'N' is the number of items and 'C' is the knapsack capacity).<br>
**Space Complexity**: $O(N * C)$. $O(N * C)$ space for the memoization array and $O(N)$ space for the recursion call-stack. So the total space complexity will be $O(N*C + N)$, which is asymptotically equivalent to $O(N*C)$.

### Bottom-up Dynamic Programming

Let’s try to populate our *dp[][]* array from the above solution by working in a bottom-up fashion. Essentially, we want to find the maximum profit for every sub-array and every possible capacity. **This means that dp[i][c] will represent the maximum knapsack profit for capacity 'c' calculated from the first 'i' items.**

So, for each item at index 'i' (0 <= i < items.length) and capacity 'c' (0 <= c <= capacity), we have two options:
1. Exclude the item at index 'i'. In this case, we will take whatever profit we get from the sub-array excluding this item => *dp[i-1][c]*
2. Include the item at index ‘i’ if its weight is not more than the capacity. In this case, we include its profit plus whatever profit we get from the remaining capacity and from remaining items => *profit[i] + dp[i-1][c-weight[i]]*

Finally, our optimal solution will be maximum of the above two values:<br>
**$dp[i][c] = max (dp[i-1][c], profit[i] + dp[i-1][c-weight[i]])$**

In [6]:
def solve_knapsack(profits, weights, capacity):
    # basic checks
    n = len(profits)
    if capacity <= 0 or n == 0 or len(weights) != n:
        return 0
    
    dp = [[0 for x in range(capacity+1)] for y in range(n)]
    
    # populate the capacity = 0 columns, with '0' capacity we have '0' profit
    for i in range(0, n):
        dp[i][0] = 0
        
    # if we have only one weight, we will take it if it is not more than the capacity
    for c in range(1, capacity+1):
        if weights[0] <= c:
            dp[0][c] = profits[0]
            
    # process all sub-arrays for all the capacities
    for i in range(1, n):
        for c in range(1, capacity+1):
            profit1, profit2 = 0, 0
            # include the item, if it is not more than the capacity
            if weights[i] <= c:
                profit1 = profits[i] + dp[i-1][c-weights[i]]
            # exclude the item
            profit2 = dp[i-1][c]
            # take maximum
            dp[i][c] = max(profit1, profit2)
            
    # maximum profit will be at the bottom-right corner.
    return dp[n-1][capacity]

def main():
    print(solve_knapsack([1, 6, 10, 16], [1, 2, 3, 5], 5))
    print(solve_knapsack([1, 6, 10, 16], [1, 2, 3, 5], 6))
    print(solve_knapsack([1, 6, 10, 16], [1, 2, 3, 5], 7))

main()

16
17
22


**Time Complexity**: $O(N * C)$, where 'N' is the number of items and 'C' is the knapsack capacity).<br>
**Space Complexity**: $O(N * C)$.

# How can we find the selected items?

As we know, the final profit is at the bottom-right corner. Therefore, we will start from there to find the items that will be going into the knapsack.

As you remember, at every step, we had two options: include an item or skip it. If we skip an item, we take the profit from the remaining items (i.e., from the cell right above it); if we include the item, then we jump to the remaining profit to find more items.

In [11]:
from __future__ import print_function

def solve_knapsack(profits, weights, capacity):
    # basic checks
    n = len(profits)
    if capacity <= 0 or n ==0 or len(weights) != n:
        return 0
    
    dp = [[0 for x in range(capacity+1)] for y in range(n)]
    
    # populate the capacity = 0 columns, with '0' capacity we have '0' profit
    for i in range(0, n):
        dp[i][0] = 0
    
    # if we have only one weight, we will take it if it is not more than the capacity
    for c in range(capacity+1):
        if weights[0] <= c:
            dp[0][c] == profits[0]
            
    # process all sub-arrays for all the capacities
    for i in range(1, n):
        for c in range(1, capacity+1):
            profit1, profit2 = 0, 0
            # include the item, if it is not more than the capacity
            if weights[i] <= c:
                profit1 = profits[i] + dp[i-1][c - weights[i]]
            # exclude the item
            profit2 = dp[i - 1][c]
            # take maximum
            dp[i][c] = max(profit1, profit2)
    print_selected_elements(dp, weights, profits, capacity)
    # maximum profit will be at the bottom-right corner.
    return dp[n - 1][capacity]

def print_selected_elements(dp, weights, profits, capacity):
    print("Selected weights are: ", end='')
    n = len(weights)
    totalProfit = dp[n-1][capacity]
    for i in range(n-1, 0, -1):
        if totalProfit != dp[i - 1][capacity]:
            print(str(weights[i]) + " ", end='')
            capacity -= weights[i]
            totalProfit -= profits[i]

    if totalProfit != 0:
        print(str(weights[0]) + " ", end='')
    print()

def main():
    print("Total knapsack profit: " +
            str(solve_knapsack([1, 6, 10, 16], [1, 2, 3, 5], 7)))
    print("Total knapsack profit: " +
            str(solve_knapsack([1, 6, 10, 16], [1, 2, 3, 5], 6)))


main()

Selected weights are: 5 2 
Total knapsack profit: 22
Selected weights are: 3 2 
Total knapsack profit: 16


### Challenge
Can we improve our bottom-up DP solution even further? Can you find an algorithm that has O(C) space complexity?

We only need one previous row to find the optimal solution!

In [19]:
def solve_knapsack(profits, weights, capacity):
    # basic checks
    n = len(profits)
    if capacity <= 0 or n == 0 or len(weights) != n:
        return 0
    
    # we only need one previous row to find the optimal solution, overall we need '2' rows
    # the above solution is similar to the previous solution, the only difference is that
    # we use `i % 2` instead if `i` and `(i-1) % 2` instead if `i-1`
    dp = [[0 for x in range(capacity+1)] for y in range(2)]
    
    # if we have only one weight, we will take it if it is not more than the capacity
    for c in range(capacity+1):
        if weights[0] <= c:
            dp[0][c] = dp[1][c] = profits[0]
            
    # process all sub-arrays for all the capacities
    for i in range(1, n):
        for c in range(0, capacity+1):
            profit1, profit2 = 0, 0
            # include the item, if it is not more than the capacity
            if weights[i] <= c:
                profit1 = profits[i] + dp[(i - 1) % 2][c - weights[i]]
            # exclude the item
            profit2 = dp[(i - 1) % 2][c]
            # take maximum
            dp[i % 2][c] = max(profit1, profit2)
    return dp[(n - 1) % 2][capacity]

def main():
    print("Total knapsack profit: " +
            str(solve_knapsack([1, 6, 10, 16], [1, 2, 3, 5], 7)))
    print("Total knapsack profit: " +
            str(solve_knapsack([1, 6, 10, 16], [1, 2, 3, 5], 6)))

main()

Total knapsack profit: 22
Total knapsack profit: 17


The solution above is similar to the previous solution; the only difference is that we use i%2 instead of i and (i-1)%2 instead of i-1. This solution has a space complexity of $O(2*C) = O(C)$, where 'C' is the knapsack’s maximum capacity.

If you see closely, we need two values from the previous iteration: dp[c] and dp[c-weight[i]]

Since our inner loop is iterating over c:0-->capacity, let’s see how this might affect our two required values:

1. When we access dp[c], it has not been overridden yet for the current iteration, so it should be fine.
2. dp[c-weight[i]] might be overridden if “weight[i] > 0”. Therefore we can’t use this value for the current iteration.

To solve the second case, we can change our inner loop to process in the reverse direction: c:capacity-->0. This will ensure that whenever we change a value in dp[], we will not need it again in the current iteration.

In [20]:
def solve_knapsack(profits, weights, capacity):
    # basic checks
    n = len(profits)
    if capacity <= 0 or n == 0 or len(weights) != n:
        return 0
    
    dp = [0 for x in range(capacity+1)]
    
    # if we have only one weight, we will take it if it is not more than the capacity
    for c in range(capacity+1):
        if weights[0] <= c:
            dp[c] = profits[0]
    
    # process all sub-arrays for all the capacities
    for i in range(1, n):
        for c in range(capacity, -1, -1):
            profit1, profit2 = 0, 0
            # include the item, if it is not more than the capacity
            if weights[i] <= c:
                profit1 = profits[i] + dp[c - weights[i]]
            # exclude the item
            profit2 = dp[c]
            # take maximum
            dp[c] = max(profit1, profit2)
    return dp[capacity]

def main():
  print("Total knapsack profit: " +
        str(solve_knapsack([1, 6, 10, 16], [1, 2, 3, 5], 7)))
  print("Total knapsack profit: " +
        str(solve_knapsack([1, 6, 10, 16], [1, 2, 3, 5], 6)))

main()

Total knapsack profit: 22
Total knapsack profit: 17
